In [1]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from guardrail import apply_guardrail

In [2]:
async def extract_text(chain, topic):
    collected_text = []
    async for event in chain.astream_events({"topic": topic}):
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if chunk.content:
                for content_item in chunk.content:
                    if text := content_item.get('text'):
                        collected_text.append(text)
                        print(text, end="", flush=True)
    return "".join(collected_text)



In [3]:
def guardrail_application(text):
    condition, violations, filtered_text, response = apply_guardrail(text ,
                                                                  "OUTPUT", 
                                                                  'us-east-1', 
                                                                  "f6563px1taqw", 
                                                                  "DRAFT"
                                                                )
    return condition, violations, filtered_text, response
    

In [4]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # replace with your region
)

In [5]:
# Another way to initialize with streaming
chat = ChatBedrockConverse(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client,
    temperature=0.7,
    max_tokens=2000
)


In [6]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{topic}")
])


In [7]:
parser = StrOutputParser()
chain = prompt | chat | parser


In [9]:
full_text = await extract_text(chain, "세계에서 유명한 CEO 10명에 대한 이름과 자세한 설명을 같이 적어줘")

세계에서 가장 유명한 CEO 10명과 그들에 대한 간략한 설명은 다음과 같습니다.

. 일론 머스크 (Elon Musk) - 테슬라, SpaceX, 뉴럴링크 등의 CEO로 전기차와 우주 탐사 분야에서 혁신적인 업적을 이뤘습니다.

.팀 쿡 (Tim Cook) - 애플의 CEO로 스티브 잡스 이후 회사를 이끌며 아이폰, 아이패드 등 혁신 제품을 지속 출시했습니다. 

.제프 베저스 (Jeff Bezos) - 아마존의 설립자이자 전 CEO로 전자상거래 혁명을 이끌었고 블루 오리진을 통해 우주 사업에도 진출했습니다.

.사티아 나델라 (Satya Nadella) - 마이크로소프트의 CEO로 클라우드 컴퓨팅과 AI 분야에서 회사를 성장시켰습니다.

.메리 바라 (Mary Barra) - 제너럴모터스의 CEO로 자동차 업계의 성공적인 여성 CEO 중 한 명입니다.

.진펭 (Ma Huateng) - 텐센트의 설립자 겸 CEO로 중국 최대 인터넷 기업을 이끌고 있습니다.

.마크 저커버그 (Mark Zuckerberg) - 페이스북의 공동 창립자이자 CEO로 소셜 미디어 혁명을 주도했습니다.

.리드 헤이스팅스 (Reed Hastings) - 넷플릭스의 공동 설립자 겸 CEO로 온라인 스트리밍 서비스를 선구적으로 이끌었습니다.

.샐리 크루 (Salli Krawcheck) - 엘레벤 전 CEO로 금융 분야에서 여성 지도자로 활약했습니다.

.아리아나 허플잉턴 (Arianna Huffington) - 허핑턴포스트의 공동 설립자이자 전 CEO로 온라인 저널리즘 혁신을 이끌었습니다.

In [21]:
from IPython.display import display, clear_output


In [22]:
stream = chain.astream_events({"topic": "세계에서 유명한 CEO 10명에 대한 이름과 자세한 설명을 같이 적어줘"})

In [23]:

async def process_stream_display(stream):
    collected_text = []
    processed_count = 0
    full_output = ""
    
    async for event in stream:
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if chunk.content:
                for content_item in chunk.content:
                    if text := content_item.get('text'):
                        collected_text.append(text)
                        
                        # 길이가 100의 배수에 도달했는지 확인
                        length_string = len(collected_text)
                        print_point = length_string % 100
                        if print_point == 0:
                            # 새로운 배치 처리 (마지막 100개 항목)
                            quotient = length_string // 100  
                            end_p = quotient*100
                            start_p = quotient*100 - 100
                            
                            current_batch = collected_text[start_p:end_p]
                            batch_text = "".join(current_batch)
                            condition, violations, filtered_text, response = guardrail_application(batch_text)
                            
                            # 출력에 추가
                            full_output += filtered_text
                            
                            # 전체 출력 업데이트
                            clear_output(wait=True)
                            display(full_output)
                            
                            # 처리된 카운트 업데이트
                            processed_count = len(collected_text)
    
    # 남은 텍스트 처리
    if len(collected_text) > processed_count:
        remaining_batch = collected_text[processed_count:]
        remaining_text = "".join(remaining_batch)
        condition, violations, filtered_text, response = guardrail_application(remaining_text)
        
        # 출력에 추가
        full_output += filtered_text
        
        # 최종 출력 업데이트
        clear_output(wait=True)
        display(full_output)


In [24]:
await process_stream_display(stream)


'물론입니다. 세계에서 가장 유명한 CEO 10명과 그들에 대한 자세한 설명을 알려드리겠습니다.\n\n1. {NAME}) - 테{NAME}, 스{NAME}, {NAME} 등의 CEO로 전기차와 우주 탐사 분야에서 혁신적인 기술을 선보이고 있습니다. 비전과 추진력이 강한 리더로 평가받고 있습니다.\n\n2. {NAME} ({NAME}) - 애플의 CEO로 {NAME} 이후 회사를 이끌고 있습니다. 아이폰, 아이패드 등 혁신적인 제품을 지속적으로 출시하며 애플의 성장을 이끌고 있습니다.\n\n3. {NAME}{NAME}) - 아마존의 창업자이자 전 CEO로 전자상거래 혁명을 이끌었습니다. 우주 탐사 기업 블루 오리진도 운영하고 있습니다.\n\n4.{NAME}{NAME}) - 페이스북의 창업자이자 CEO로 소셜미디어 분야를 선도하고 있습니다. 메타버스에도 많은 투자를 하고 있습니다.\n\n5.{NAME}) - 마이크로소프트의 CEO로 클라우드 컴퓨팅과 인공지능 분야에서 적극적인 투자를 하고 있습니다.\n\n6.{NAME}) - 넷플릭스의 공동 창업자이자 CEO로 온라인 스트리밍 서비스를 선도하고 있습니다.\n\n7.{NAME}) - 마이크로소프트의 전 CEO로 기업 문화 혁신과 성장에 기여했습니다.\n\n8.{NAME} ({NAME}) - 제너럴모터스의 CEO로 자동차 산업에서 여성 CEO로는 드물게 성공을 거두고 있습니다. \n\n9.{NAME}{NAME}) - 스타벅스의 전 CEO로 커피 전문점 체인을 세계적인 브랜드로 성장시켰습니다.\n\n10. {NAME}) - 구글의 CEO로 인공지능과 클라우드 컴퓨팅 분야에서 혁신을 이끌고 있습니다.'